<a href="https://colab.research.google.com/github/soumyadg/LLMs/blob/main/Incontext_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken chromadb pypdf pypdf2

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.233
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
!wget -q https://www.dropbox.com/scl/fo/homeoxhk7gueaj019dbix/h?rlkey=2etgal599klnahhem9mmdakqg&dl=0
!unzip -q new_articles.zip -d new_articles

replace new_articles/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
import openai
import os

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# Load and process the text files
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [ ]:
# # Load and process the text files
# loader = DirectoryLoader('./new_articles/', glob="./*.pdf", loader_cls=PyPDFLoader)
# documents = loader.load()
# documents

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

233

In [ ]:
texts[3]

Document(page_content='Checks leans on artificial intelligence and machine learning to scan apps and their code to identify areas where there might be violations of privacy and data protection rules, and provides remediation to suggest how to fix it — tasks that would be far more difficult for a team of humans to execute on their own. It’s already integrated with Google’s large language models and what it describes as “app understanding technologies” to power what it identifies and make suggestions for fixing issues.\n\nA dashboard lets users monitor and triage issues in the areas of compliance monitoring, data monitoring and store disclosure support (which is focused specifically on Google Play data safety). With the service also aimed at iOS developers, it’s not clear if it will add Apple App Store data safety at any point into that mix. All of this can be monitored in real time on live apps, as well as when they are still in development.', metadata={'source': 'new_articles/05-03-che

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,embedding=embedding,persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,embedding_function=embedding)

In [ ]:
## Make a Retriver

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("How much money did Pando raise?")

In [ ]:
len(docs)

4

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
retriever.search_type

'similarity'

In [ ]:
retriever.search_kwargs

{'k': 2}

In [ ]:
## Make a Chain

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [ ]:
# break it down
query = "What is the news about Pando?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'What is the news about Pando?',
 'result': ' Pando is a software-as-a-service platform offering that was co-launched by Jayakrishnan and Abhijeet Manohar in 2018 to solve for global logistics. They saw firsthand that manufacturers, distributors and retailers were struggling with legacy tech and point solutions to understand, optimize and manage their global logistics operations.',
 'source_documents': [Document(page_content='Pando was co-launched by Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery, an India-based freight tech marketplace — and their first startup. The two saw firsthand manufacturers, distributors and retailers were struggling with legacy tech and point solutions to understand, optimize and manage their global logistics operations — or at least, that’s the story Jayakrishnan tells.\n\n“Supply chain leaders were trying to build their own tech and throwing people at the problem,” he said. “This caught our attention — we spent month

In [ ]:
query = "Who led the round in Pando?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Iron Pillar and Uncorrelated Ventures led the round.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [ ]:
# Deleting the DB

In [ ]:
!zip -r db.zip ./db


updating: db/ (stored 0%)
updating: db/chroma-embeddings.parquet (deflated 29%)
updating: db/chroma-collections.parquet (deflated 50%)
updating: db/index/ (stored 0%)
updating: db/index/id_to_uuid_d678244f-bb50-46e9-8330-0baf1cfc3b9b.pkl (deflated 35%)
updating: db/index/index_metadata_d678244f-bb50-46e9-8330-0baf1cfc3b9b.pkl (deflated 14%)
updating: db/index/index_d678244f-bb50-46e9-8330-0baf1cfc3b9b.bin (deflated 17%)
updating: db/index/uuid_to_id_d678244f-bb50-46e9-8330-0baf1cfc3b9b.pkl (deflated 39%)
updating: db/index/uuid_to_id_c4a1eba8-04c3-4f18-84e0-1dc5959f0b5b.pkl (deflated 39%)
updating: db/index/id_to_uuid_c4a1eba8-04c3-4f18-84e0-1dc5959f0b5b.pkl (deflated 35%)
updating: db/index/index_metadata_c4a1eba8-04c3-4f18-84e0-1dc5959f0b5b.pkl (deflated 11%)
updating: db/index/index_c4a1eba8-04c3-4f18-84e0-1dc5959f0b5b.bin (deflated 17%)


In [ ]:
# create the chain to answer questions - combine the retrival part and the generation part
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
# cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
def chat_gpt_response(message):

    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        temperature = 0,
        messages = [
            {"role": "user", "content": message}
        ]
    )
    return response.choices[0]["message"]["content"]

In [ ]:
#@title
# RAG
query = "What type of contracts are these?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 These are brand-related security contracts.


Sources:
new_articles/05-03-nova-is-building-guardrails-for-generative-ai-content-to-protect-brand-integrity.txt
new_articles/05-03-nova-is-building-guardrails-for-generative-ai-content-to-protect-brand-integrity.txt


In [ ]:
# Question 1

In [ ]:
# RAG
query = "whats with Elon Musk and Randeep"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Elon Musk and Randeep Hothi were involved in a defamation case, which recently ended with Musk paying Hothi 10 million dollars to settle the case.


Sources:
new_articles/05-06-amazon-launches-free-channels-check-marks-come-to-gmail-and-openai-raises-more-moolah.txt
new_articles/05-06-amazon-launches-free-channels-check-marks-come-to-gmail-and-openai-raises-more-moolah.txt


In [ ]:
# print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

In [ ]:
# ChatGPT
chat_gpt_response(query)

'There is no known connection between Elon Musk, the CEO of Tesla and SpaceX, and Randeep, as the name "Randeep" does not provide enough information to identify a specific individual. It is possible that you may be referring to someone else or have misspelled the name.'

In [ ]:
# RAG
query = "whats is prometheus"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Prometheus is not mentioned in the context, so I don't know.


Sources:
new_articles/05-05-vint-cerf-on-the-exhilarating-mix-of-thrill-and-hazard-at-the-frontiers-of-tech.txt
new_articles/05-05-vint-cerf-on-the-exhilarating-mix-of-thrill-and-hazard-at-the-frontiers-of-tech.txt


In [ ]:
# CHAT GPT
chat_gpt_response(query)

'Prometheus is a figure in Greek mythology who is known for his role in the creation of humanity and his defiance of the gods. According to the myth, Prometheus stole fire from the gods and gave it to humans, which allowed them to progress and develop civilization. As a punishment for his actions, Zeus, the king of the gods, chained Prometheus to a rock where an eagle would eat his liver every day, only for it to regenerate overnight. Prometheus is often seen as a symbol of human progress, rebellion, and the consequences of challenging divine authority. The myth of Prometheus has been referenced and interpreted in various works of literature, art, and popular culture.'